In [1]:
import pandas as pd
import numpy as np
import random as rd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.graph_objects as go
import seaborn as sns
import networkx as nx
import plotly.express as px
import plotly.offline as py
from sklearn.preprocessing import LabelEncoder

sns.axes_style("whitegrid")
sns.set(rc = {'figure.figsize': (15, 8)})

import warnings
warnings.filterwarnings('ignore')

Using the Airlines Data, visualize and analyze the

1. number of visits per city using geospatial visualization

2.  the relationship and frequency of flights using network visualization

In [2]:
edges = pd.read_csv('../datasets/airlines_edges.csv')
print(edges.shape)
# display(edges.head())

nodes = pd.read_csv('../datasets/airlines_nodes.csv')
print(nodes.shape)
# display(nodes.head())

(1297, 3)
(235, 8)


## GEOSPATIAL VISUALIZATION

In [3]:
choropleth = px.scatter_geo(data_frame=nodes, lat='latitude', lon='longitude', locationmode='USA-states', geojson=None, 
    color="Visits", size="Visits", scope='usa', center=None, width=None, height=None, hover_data=['City', 'Label'],
    title='Geospatial Visualization on Number of Visits per City')
choropleth.update_layout(margin=dict(l=5, r=5, t=80, b=20))
choropleth.show()

## NETWORK VISUALIZATION

In [4]:
# Transform nodes to dictionary
nodes_dict = nodes.set_index('ID')['Label'].to_dict()

# Rename edges based on dictionary
edges_cleaned = [edges[x].map(nodes_dict) for x in edges.columns if x != 'Freq']
edges_cleaned = pd.DataFrame(edges_cleaned).T
edges_cleaned['Freq'] = edges['Freq']
# display(edges_cleaned.head())

In [5]:
# Clean the dataset for visualization
rt = pd.merge(edges_cleaned, nodes, left_on='Source', right_on='Label')
rt = pd.merge(rt, nodes, left_on='Target', right_on='Label').drop_duplicates().reset_index(drop=True)
max_count = max(rt['Freq'])
# display(rt.head())

In [ ]:
# Airports initialization
airports_graph = [dict(
        type = 'scattergeo',
        locationmode = 'country names',
        lat = rt['latitude_x'],
        lon = rt['longitude_x'],
        text = '<b>' + rt['Source'].map(str) + '</b>' \
        + '<br>' + 'Frequency: ' + rt['Freq'].map(str) \
        + '<br>' + 'City: ' + rt['City_x'].map(str) \
        + '<br>' + 'Visits: ' + rt['Visits_x'].map(str),
        hoverinfo = 'text',
        marker=dict(
            size=np.array(rt.Freq*20/max(rt.Freq)),
            sizemin=2,
            color = np.array(rt.Freq),
            colorscale = [[0,'rgb(255, 255, 0)'],[1,'rgb(255, 0, 0)']],
            colorbar=go.ColorBar(title='Frequency of Flights')
        ),
        opacity = 1,
        line = dict(
            width=10,
            color='rgba(0, 100, 255, 255)'
        )          
    )]

In [16]:
# Flight paths initialization
fp = []
for x in range(len(rt)):
    fp.append(
        dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lat = [rt['latitude_x'][x], rt['latitude_y'][x]],
            lon = [rt['longitude_x'][x], rt['longitude_y'][x]],
            mode = 'lines',
            hoverinfo = 'skip',
            line = dict(
                width = 0.3,
                color = 'rgba(0, 0, 255, 255)'
            ),
            opacity = float(rt.loc[x, 'Freq']/max_count)
        )
    )

In [17]:
# Layout initialization
layout = go.Layout(
        showlegend = False, 
        geo = dict(
            scope='usa',
            showland = True,
            showocean = True,
            showcoastlines = True,
            showcountries = True,           
            landcolor = 'rgb(49, 49, 49)',
            countrycolor = 'rgb(90, 90, 90)',
            coastlinecolor = 'rgb(90, 90, 90)',
            oceancolor = 'rgb(29, 29, 29)',
            bgcolor = 'rgb(29, 29, 29)'
        ),
        margin=dict(l=10, r=10, t=80, b=20),
        autosize=True,
        paper_bgcolor = 'rgb(29, 29, 29)',
        plot_bgcolor = 'rgb(29, 29, 29)')

In [18]:
# Show airports and flight paths visualization
fig = dict(data=fp+airports_graph, layout=layout)
py.iplot(fig, filename='routes-graph')